In [1]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-u4dddw76
  Stored in directory: /tmp/pip-ephem-wheel-cache-_2bm1wpi/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
directory /home/nineleaps/Desktop/src already exists
Out bin /home/nineleaps/Desktop/result.out


Q 1

In [ ]:
%%cu
#include<stdio.h>
__managed__ int sum=0;
__global__ void Array_sum(int *a,int *n)
{
    int tid=threadIdx.x;
    if(tid<*n)
      atomicAdd(&sum,a[tid]);
}

int main()
{
    int n=10,i;
    //printf("Enter N:");
    //scanf("%d",&n);
    int a[n];
    int *cuda_a,*cuda_n;
 
    for(i=0;i<n;i++)
    {
        a[i]=rand()%100;
        printf("%d ",a[i]);
    }
    printf("\n");

    cudaMalloc((void**)&cuda_a,n*sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    
    cudaMemcpy(cuda_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    Array_sum <<<1,n>>>(cuda_a,cuda_n);
    printf("Sum:%d\n",sum);
    cudaFree(cuda_a);
    cudaFree(cuda_n);
    return 0;
}

8 6 7 1 9 3 8 9 4 2 

Sum:56

Q 2

In [ ]:
%%cu
#include<stdio.h>

__global__ void Array_add(int *a,int *b,int *c,int *n)
{
    unsigned short tid=threadIdx.x;
    if(tid<*n)
      c[tid]=a[tid]+b[tid];
}

int main()
{
    int n=5,i;
    //printf("Enter N:");
    //scanf("%d",&n);
    int a[n],b[n],c[n];
    int *cuda_a,*cuda_b,*cuda_c,*cuda_n;
 
    for(i=0;i<n;i++)
      a[i]=rand()%100;
    for(i=0;i<n;i++)
      b[i]=rand()%100;

    cudaMalloc((void**)&cuda_a,n*sizeof(int));
    cudaMalloc((void**)&cuda_b,n*sizeof(int));
    cudaMalloc((void**)&cuda_c,n*sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    
    cudaMemcpy(cuda_a,a,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b,b,n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    Array_add <<<1,n>>>(cuda_a,cuda_b,cuda_c,cuda_n);
    cudaMemcpy(c,cuda_c,n*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<n;i++)
      printf("%d + %d = %d\n",a[i],b[i],c[i]);
    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_c);
    cudaFree(cuda_n);
    return 0;
}

8 + 3 = 11 

8 + 6 = 14 

7 + 2 = 9 

1 + 4 = 5 

9 + 1 = 10

Q 3

In [ ]:
%%cu
#include<stdio.h>

__global__ void Matrix_mult(int *a,int *b,int *c,int *m,int *n,int *p)
{
    int col=blockIdx.y*blockDim.y+threadIdx.y;
    int row=blockIdx.x*blockDim.x+threadIdx.x;
    int temp=0,i;

    if(row<*m&&col<*p)
    for(i=0;i<*n;i++)
      temp+=a[row*(*n)+i]*b[i*(*p)+col];
    c[row*(*p)+col]=temp;
}

int main()
{
    int m=3,n=2,p=2,i,j;
    //printf("Enter N:");
    //scanf("%d",&n);
    int a[m*n],b[n*p],c[m*n];
    int *cuda_a,*cuda_b,*cuda_c,*cuda_m,*cuda_n,*cuda_p;
    printf("Matrix A:\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<n;j++)
        {
            a[i*n+j]=rand()%100;
            printf("%d ",a[i*n+j]);       
        }
        printf("\n");
    }
    printf("\nMatrix B:\n");
    for(i=0;i<n;i++)
    {
        for(j=0;j<p;j++)
        {
            b[i*p+j]=rand()%100;
            printf("%d ",b[i*p+j]);       
        }
        printf("\n");
    }
    printf("\n");

    cudaMalloc((void**)&cuda_a,m*n*sizeof(int));
    cudaMalloc((void**)&cuda_b,n*p*sizeof(int));
    cudaMalloc((void**)&cuda_c,m*p*sizeof(int));
    cudaMalloc((void**)&cuda_m,sizeof(int));
    cudaMalloc((void**)&cuda_n,sizeof(int));
    cudaMalloc((void**)&cuda_p,sizeof(int));

    cudaMemcpy(cuda_a,a,m*n*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_b,b,n*p*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_m,&m,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_n,&n,sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(cuda_p,&p,sizeof(int),cudaMemcpyHostToDevice);
 
    dim3 threadsPerBlock(m,p);
    dim3 blocksPerGrid(1,1);
        

    Matrix_mult<<<blocksPerGrid,threadsPerBlock>>> (cuda_a,cuda_b,cuda_c,cuda_m,cuda_n,cuda_p);   

    cudaMemcpy(c,cuda_c,m*p*sizeof(int),cudaMemcpyDeviceToHost);
    printf("Result Matrix:\n");
    for(i=0;i<m;i++)
    {
        for(j=0;j<p;j++)
        {
            printf("%d ",c[i*p+j]);       
        }
        printf("\n");
    }
    cudaFree(cuda_a);
    cudaFree(cuda_b);
    cudaFree(cuda_c);
    cudaFree(cuda_m);
    cudaFree(cuda_n);
    cudaFree(cuda_p);
    return 0;
}

Matrix A:

1 2

3 4

Matrix B :

1 1

1 1

Result matrix

3 3

7 7

Q 7

In [ ]:
%%cu
#include<thrust/host_vector.h>
#include<thrust/device_vector.h>
#include<thrust/generate.h>
#include<thrust/sort.h>
#include<thrust/copy.h>
#include<cstdlib>

int main()
{
    thrust::host_vector<int> a(10);
    thrust::generate(a.begin(),a.end(),rand);
    thrust::device_vector<int> device_a=a;
    thrust::sort(device_a.begin(),device_a.end());
    thrust::copy(device_a.begin(),device_a.end(),a.begin());
    for(auto x:a)
      printf("%d ",x);
    return 0;
}

424 596 719 846 118 164 168 171 180 195